prompt for task 1: Task: Connect to a Movie Database API (e.g., OMDb or TMDB) to fetch details of a movie. Generate Python code to query the API by movie title. Handle errors like invalid movie name, missing/expired API key, and timeout. Display title, release year, genre, IMDb rating, and director in a clean format. Use an environment variable for the API key and provide a clear fallback/error messages.

In [ ]:
"""
task_movie_lookup.py

Usage:
- Set OMDB_API_KEY in your environment, or the script will prompt you for one.
- Run: python task_movie_lookup.py
- Example: enter "The Matrix" when prompted for a title.

This script uses the OMDb API: http://www.omdbapi.com/
"""

import os
import requests
import sys
from requests.exceptions import RequestException, Timeout

OMDB_DEFAULT_URL = "http://www.omdbapi.com/"

def get_api_key():
    key = os.environ.get("OMDB_API_KEY")
    if key:
        return key.strip()
    # Prompt user (useful in Colab/interactive)
    key = input("Enter OMDb API key (or press Enter to abort): ").strip()
    if not key:
        print("API key required to use OMDb. Aborting.")
        sys.exit(1)
    return key

def fetch_movie(title, api_key, timeout_sec=6):
    """
    Fetch movie details from OMDb by title.
    Returns parsed JSON on success, or raises ValueError on known errors.
    """
    params = {"t": title, "apikey": api_key, "r": "json"}
    try:
        resp = requests.get(OMDB_DEFAULT_URL, params=params, timeout=timeout_sec)
    except Timeout:
        raise Timeout("Request timed out")
    except RequestException as e:
        raise ConnectionError(f"Network error: {e}") from e

    # OMDb always returns 200 (often) but with {"Response":"False", "Error":"..."} for errors
    if resp.status_code != 200:
        raise ConnectionError(f"HTTP {resp.status_code}: {resp.text[:200]}")

    try:
        data = resp.json()
    except ValueError:
        raise ValueError("Invalid JSON response from OMDb")

    if data.get("Response", "").lower() == "false":
        # could be Invalid API key, Movie not found!, etc.
        err = data.get("Error", "Unknown error from OMDb")
        raise ValueError(f"OMDb error: {err}")

    return data

def display_movie(data):
    """
    Displays the requested fields in a clean format.
    Fields: Title, Year, Genre, imdbRating, Director
    """
    title = data.get("Title", "N/A")
    year = data.get("Year", "N/A")
    genre = data.get("Genre", "N/A")
    imdb = data.get("imdbRating", "N/A")
    director = data.get("Director", "N/A")

    print("\n=== Movie Details ===")
    print(f"Title      : {title}")
    print(f"Year       : {year}")
    print(f"Genre      : {genre}")
    print(f"IMDb Rating: {imdb}")
    print(f"Director   : {director}\n")

def main():
    api_key = get_api_key()
    title = input("Enter movie title to search: ").strip()
    if not title:
        print("Empty title provided. Aborting.")
        return

    try:
        data = fetch_movie(title, api_key)
        display_movie(data)
    except Timeout:
        print("Request timed out. Try again later or increase timeout.")
    except ValueError as ve:
        # includes OMDb 'Movie not found!' and 'Invalid API key!' messages
        print(f"Error: {ve}")
    except ConnectionError as ce:
        print(f"Connection error: {ce}")
    except Exception as e:
        print(f"Unexpected error: {e}")

if __name__ == "__main__":
    main()


Enter OMDb API key (or press Enter to abort): 123456
Enter movie title to search: sdbckc
Connection error: HTTP 401: {"Response":"False","Error":"Invalid API key!"}


prompt for task 2:
Task: Use a Public Transport API to fetch next 5 arrivals for a given stop/station ID. Handle invalid station codes, unavailable service, and malformed responses. If a real API (TransportAPI or other) credentials are not provided, the script should automatically fall back to a mock mode with sample data. Display results as a readable table: route number, destination, arrival time.

In [ ]:
"""
task_public_transport.py

Notes:
- Some public transport APIs require keys (e.g., TransportAPI, city-specific APIs).
- This script supports:
  1) Real API mode: provide a base URL and credentials via environment variables (see below).
  2) Mock mode: if no credentials provided, returns sample data for development/testing.

Environment variables (optional for real mode):
- PT_BASE_URL   : base URL of the transport API (e.g., https://transportapi.example)
- PT_APP_ID     : app id / client id if required
- PT_APP_KEY    : app key if required

Usage:
- Run and input a stop/station code when prompted.
"""

import os
import requests
from requests.exceptions import RequestException, Timeout
from datetime import datetime
from prettytable import PrettyTable  # pip install prettytable if missing

# Default timeout for HTTP calls
HTTP_TIMEOUT = 6

def mock_arrivals(stop_id, limit=5):
    # Sample mocked response structure (keeps same fields as typical APIs)
    sample = [
        {"route":"5A", "destination":"Central Station", "arrival":"2025-11-11T14:25:00"},
        {"route":"12", "destination":"West End", "arrival":"2025-11-11T14:30:00"},
        {"route":"3B", "destination":"Airport", "arrival":"2025-11-11T14:35:00"},
        {"route":"7", "destination":"City Park", "arrival":"2025-11-11T14:42:00"},
        {"route":"21", "destination":"Harbour", "arrival":"2025-11-11T14:50:00"},
    ]
    return sample[:limit]

def fetch_arrivals_real(base_url, stop_id, app_id=None, app_key=None, limit=5):
    """
    Example generic call — actual query parameters differ between transport APIs.
    This function demonstrates how to call a real API; adjust path/params to your provider.
    """
    # Example endpoint pattern (provider-specific): /stops/{stop_id}/arrivals
    url = f"{base_url.rstrip('/')}/stops/{stop_id}/arrivals"
    params = {}
    if app_id: params['app_id'] = app_id
    if app_key: params['app_key'] = app_key
    try:
        r = requests.get(url, params=params, timeout=HTTP_TIMEOUT)
    except Timeout:
        raise Timeout("Transport API request timed out")
    except RequestException as e:
        raise ConnectionError(f"Network error contacting transport API: {e}") from e

    if r.status_code == 404:
        raise ValueError("Stop/station not found (404)")
    if r.status_code >= 400:
        raise ConnectionError(f"Transport API returned HTTP {r.status_code}: {r.text[:200]}")

    try:
        payload = r.json()
    except ValueError:
        raise ValueError("Malformed JSON from transport API")

    # The following parsing is generic — adapt to the actual API response structure.
    # Try common keys, otherwise raise.
    # Example expected: payload['arrivals'] -> list of {route, destination, expected_time}
    arrivals = None
    for candidate in ('arrivals','departures','services','results'):
        if candidate in payload and isinstance(payload[candidate], list):
            arrivals = payload[candidate]
            break
    if arrivals is None:
        # Try if payload is already a list
        if isinstance(payload, list):
            arrivals = payload
        else:
            raise ValueError("Unexpected transport API response structure")

    # Map to uniform objects; try multiple possible field names
    normalized = []
    for item in arrivals[:limit]:
        # Best-effort mapping
        route = item.get('line') or item.get('route') or item.get('service') or item.get('number') or 'N/A'
        dest = item.get('destination') or item.get('towards') or item.get('headsign') or 'N/A'
        # expected/arrival time
        arr = item.get('expected') or item.get('expected_time') or item.get('arrival') or item.get('expectedArrival') or item.get('time') or None
        normalized.append({"route": route, "destination": dest, "arrival": arr})
    return normalized

def display_table(arrivals):
    table = PrettyTable()
    table.field_names = ["Route", "Destination", "Arrival (local)"]
    for a in arrivals:
        # Attempt to pretty-print arrival time
        arr_str = a.get('arrival') or 'N/A'
        try:
            # If ISO datetime string, convert to readable local time
            dt = datetime.fromisoformat(arr_str)
            arr_local = dt.strftime("%Y-%m-%d %H:%M:%S")
        except Exception:
            arr_local = arr_str
        table.add_row([a.get('route'), a.get('destination'), arr_local])
    print(table)

def main():
    base_url = os.environ.get("PT_BASE_URL")
    app_id = os.environ.get("PT_APP_ID")
    app_key = os.environ.get("PT_APP_KEY")

    stop = input("Enter stop/station id: ").strip()
    if not stop:
        print("No stop id provided. Aborting.")
        return

    # If base_url missing, fallback to mock mode
    if not base_url or not (app_id and app_key):
        print("No transport API credentials detected — running in MOCK mode with sample data.")
        arrivals = mock_arrivals(stop, limit=5)
        display_table(arrivals)
        return

    # Try real API
    try:
        arrivals = fetch_arrivals_real(base_url, stop, app_id, app_key, limit=5)
        if not arrivals:
            print("No upcoming arrivals found for this stop.")
            return
        display_table(arrivals)
    except Timeout:
        print("Request timed out. Try again later.")
    except ValueError as ve:
        print(f"Bad response or invalid stop code: {ve}")
    except ConnectionError as ce:
        print(f"Connection error: {ce}")
    except Exception as e:
        print(f"Unexpected error: {e}")

if __name__ == "__main__":
    main()

prompt for task 3:
Task: Connect to a stock data API to fetch daily stock prices. Generate a Python function that queries stock data by ticker symbol, handles API limits/invalid tickers/null responses, and displays opening price, closing price, high, low, and trading volume. Use yfinance when available; otherwise fallback to Alpha Vantage if ALPHA_VANTAGE_KEY is provided. Provide clear error handling.

In [ ]:
"""
task_stock_data.py

This script tries to use yfinance (no API key required). If yfinance is not available
or fails, and you have an Alpha Vantage API key in ALPHA_VANTAGE_KEY environment variable,
it will fallback to Alpha Vantage.

Install dependencies if needed:
- pip install yfinance requests
"""

import os
import sys
import time
from pprint import pprint

HTTP_TIMEOUT = 8

def fetch_with_yfinance(ticker):
    try:
        import yfinance as yf
    except Exception as e:
        raise ImportError("yfinance is not installed") from e

    try:
        stock = yf.Ticker(ticker)
        # fetch last trading day's data (history)
        hist = stock.history(period="5d", interval="1d")
        if hist.empty:
            raise ValueError("No data found for ticker (yfinance returned empty).")
        # Take most recent row
        last = hist.iloc[-1]
        result = {
            "date": str(last.name.date()),
            "open": float(last['Open']),
            "close": float(last['Close']),
            "high": float(last['High']),
            "low": float(last['Low']),
            "volume": int(last['Volume'])
        }
        return result
    except Exception as e:
        raise RuntimeError(f"yfinance error: {e}") from e

def fetch_with_alpha_vantage(ticker, api_key):
    import requests
    url = "https://www.alphavantage.co/query"
    params = {"function": "TIME_SERIES_DAILY_ADJUSTED", "symbol": ticker, "apikey": api_key, "outputsize": "compact"}
    try:
        r = requests.get(url, params=params, timeout=HTTP_TIMEOUT)
    except Exception as e:
        raise ConnectionError(f"Network error: {e}") from e

    if r.status_code != 200:
        raise ConnectionError(f"HTTP {r.status_code}: {r.text[:200]}")
    try:
        payload = r.json()
    except Exception:
        raise ValueError("Invalid JSON from Alpha Vantage")

    # Handle API limit or errors
    if "Error Message" in payload:
        raise ValueError("Alpha Vantage: Invalid ticker or API error.")
    if "Note" in payload:
        raise RuntimeError(f"Alpha Vantage rate limit or note: {payload['Note'][:200]}")

    ts = payload.get("Time Series (Daily)")
    if not ts:
        raise ValueError("No 'Time Series (Daily)' in Alpha Vantage response.")

    # Get most recent day
    dates = sorted(ts.keys(), reverse=True)
    latest = ts[dates[0]]
    return {
        "date": dates[0],
        "open": float(latest["1. open"]),
        "high": float(latest["2. high"]),
        "low": float(latest["3. low"]),
        "close": float(latest["4. close"]),
        "volume": int(latest["6. volume"])
    }

def get_stock_data(ticker):
    ticker = ticker.strip().upper()
    if not ticker:
        raise ValueError("Empty ticker")

    # Try yfinance first
    try:
        data = fetch_with_yfinance(ticker)
        data["source"] = "yfinance"
        return data
    except ImportError:
        # Not installed — try alpha vantage if key present
        pass
    except Exception as e:
        # yfinance failed — we'll log and try fallback
        last_err = e
        print(f"Warning: yfinance failed: {e}")

    av_key = os.environ.get("ALPHA_VANTAGE_KEY")
    if av_key:
        try:
            data = fetch_with_alpha_vantage(ticker, av_key)
            data["source"] = "alpha_vantage"
            return data
        except Exception as e:
            raise RuntimeError(f"Alpha Vantage fallback failed: {e}") from e
    else:
        # No fallback available
        raise RuntimeError("No working data source: yfinance failed or not installed, and ALPHA_VANTAGE_KEY not provided.")

def pretty_print_stock(data):
    print("\n=== Stock data ===")
    print(f"Date   : {data.get('date')}")
    print(f"Source : {data.get('source')}")
    print(f"Open   : {data.get('open')}")
    print(f"High   : {data.get('high')}")
    print(f"Low    : {data.get('low')}")
    print(f"Close  : {data.get('close')}")
    print(f"Volume : {data.get('volume')}\n")

if __name__ == "__main__":
    ticker = input("Enter ticker symbol (e.g., AAPL): ").strip()
    try:
        info = get_stock_data(ticker)
        pretty_print_stock(info)
    except ValueError as ve:
        print(f"Input/validation error: {ve}")
    except RuntimeError as re:
        print(f"Runtime error: {re}")
    except Exception as e:
        print(f"Unexpected error: {e}")

prompt for task 4:
Task: Build a translator using a free Translation API (LibreTranslate). Accept input text and target language from the user, handle invalid language codes, API quota exceeded, and empty text input, and implement a retry mechanism (3 attempts with exponential backoff). Display original and translated text.

In [ ]:
"""
task_translate.py

Uses LibreTranslate public instance by default (https://libretranslate.de).
No API key required for many public instances, but they may rate-limit.
You may set TRANSLATE_BASE_URL env var to point to another instance.

Install:
- pip install requests
"""

import os
import requests
import time
from requests.exceptions import RequestException, Timeout

BASE_URL = os.environ.get("TRANSLATE_BASE_URL", "https://libretranslate.de")
TRANSLATE_ENDPOINT = "/translate"
LANGS_ENDPOINT = "/languages"
HTTP_TIMEOUT = 8

def list_supported_languages():
    try:
        r = requests.get(BASE_URL + LANGS_ENDPOINT, timeout=HTTP_TIMEOUT)
        r.raise_for_status()
        langs = r.json()
        return {item['code']: item['name'] for item in langs}
    except Exception:
        return None  # ignore failures; we'll handle at runtime

def translate_text(text, target_lang, retries=3):
    if not text or not text.strip():
        raise ValueError("Empty text provided for translation")

    payload = {"q": text, "target": target_lang}
    headers = {"Content-Type": "application/json"}
    attempt = 0
    backoff = 1.0
    while attempt < retries:
        try:
            r = requests.post(BASE_URL + TRANSLATE_ENDPOINT, json=payload, headers=headers, timeout=HTTP_TIMEOUT)
            # Some instances respond 429 for rate-limited requests
            if r.status_code == 429:
                raise RuntimeError("Rate limit / quota exceeded (HTTP 429)")
            r.raise_for_status()
            data = r.json()
            # LibreTranslate returns {'translatedText': '...'}
            translated = data.get("translatedText") or data.get("translation") or data.get("result") or None
            if not translated:
                # Unexpected response structure
                raise ValueError("Unexpected translation response")
            return translated
        except (RequestException, Timeout) as e:
            # Network-related, retry
            attempt += 1
            if attempt >= retries:
                raise ConnectionError(f"Network request failed after {attempt} attempts: {e}")
            time.sleep(backoff)
            backoff *= 2
        except RuntimeError as re:
            # e.g., 429
            attempt += 1
            if attempt >= retries:
                raise
            time.sleep(backoff)
            backoff *= 2
        except ValueError as ve:
            # non-recoverable or malformed response
            raise

def main():
    # Show supported languages if possible
    langs = list_supported_languages()
    if langs:
        print("Supported language codes (sample):", ", ".join(list(langs.keys())[:10]), "...")
    else:
        print("Could not fetch supported languages list. Proceeding; ensure you use valid language codes (e.g., 'es' for Spanish, 'fr' for French).")

    text = input("Enter text to translate: ").strip()
    if not text:
        print("Empty text — aborting.")
        return
    target = input("Enter target language code (e.g., 'es', 'fr', 'hi'): ").strip()
    if not target:
        print("No target language provided. Aborting.")
        return

    # Basic validation if languages list fetched
    if langs and target not in langs:
        print(f"Warning: language code '{target}' not in fetched list. You may still try, but it could fail.")

    try:
        translated = translate_text(text, target, retries=3)
        print("\n--- TRANSLATION RESULT ---")
        print("Original :", text)
        print("Translated:", translated)
    except ValueError as ve:
        print(f"Bad response / input error: {ve}")
    except ConnectionError as ce:
        print(f"Network error: {ce}")
    except RuntimeError as re:
        print(f"API error: {re}")
    except Exception as e:
        print(f"Unexpected error: {e}")

if __name__ == "__main__":
    main()